In [1]:
import sys
import numpy as np
import pandas as pd
from scipy import stats
pd.options.mode.chained_assignment = None 

In [2]:
df_category = pd.read_excel('https://www.dropbox.com/s/jcdjj2znimqbico/category.xlsx?dl=1')
df_sales = pd.read_excel('https://www.dropbox.com/s/7359pamv5jcq1cl/sales.xlsx?dl=1')

In [3]:
def keep_cols(DataFrame, keep_these):
    drop_these = list(set(list(DataFrame)) - set(keep_these))
    return DataFrame.drop(drop_these, axis = 1)

In [4]:
df_sales = df_sales.pipe(keep_cols, ['UPC', 'BASE_PRICE', 'PRICE', 'UNITS'])
df_category = df_category.pipe(keep_cols, ['UPC', 'CATEGORY'])

In [5]:
df_sales.head()

,UPC,UNITS,PRICE,BASE_PRICE
0,1111009477,13,1.39,1.57
1,1111009497,20,1.39,1.39
2,1111009507,14,1.38,1.38
3,1111035398,4,3.50,4.49
4,1111038078,3,2.50,2.50


In [6]:
unique_upc = df_sales['UPC'].unique()

In [7]:
corr_df = df_sales[df_sales['UPC'] == unique_upc[2]]
del corr_df['UPC']
corr_df.corr()

,UNITS,PRICE,BASE_PRICE
UNITS,1.000000,0.319743,0.378383
PRICE,0.319743,1.000000,0.888873
BASE_PRICE,0.378383,0.888873,1.000000


In [8]:
def get_optimal_price(current_df):
    current_df['demand'] = current_df['BASE_PRICE'] - current_df['PRICE']
    df = current_df[current_df['demand'] > 0]
    df['profit'] = df['demand'] * df['UNITS']
    df['zscore'] = np.abs(stats.zscore(df['PRICE']))
    df = df[df['zscore'] < 3]
    df_std = np.std(df['PRICE']) / 2
    index_max = df['profit'].idxmax()
    df_max = df.loc[index_max, 'PRICE']
    price_arr = df[(df['PRICE'] > df_max - df_std) & (df['PRICE'] < df_max + df_std)]['PRICE']
    return np.mean(price_arr)

In [9]:
upc_df = pd.DataFrame(columns=['upc', 'price'])
for upc in unique_upc:
    df = df_sales[df_sales['UPC'] == upc]
    upc_df = upc_df.append({
        'upc': str(upc), 
        'price': get_optimal_price(df)
    }, ignore_index=True)

In [10]:
upc_df

,upc,price
0,1111009477,1.230438
1,1111009497,1.233123
2,1111009507,1.237765
3,1111035398,2.641161
4,1111038078,0.989248
5,1111038080,0.986411
6,1111085319,1.631855
7,1111085345,1.642604
8,1111085350,1.894247
9,1111087395,2.947300


In [11]:
full_df = pd.merge(df_sales, df_category, on='UPC')

In [12]:
full_df.head()

,UPC,UNITS,PRICE,BASE_PRICE,CATEGORY
0,1111009477,13,1.39,1.57,BAG SNACKS
1,1111009477,111,1.39,1.39,BAG SNACKS
2,1111009477,71,1.39,1.39,BAG SNACKS
3,1111009477,76,1.23,1.23,BAG SNACKS
4,1111009477,33,1.16,1.16,BAG SNACKS


In [13]:
unique_category = full_df['CATEGORY'].unique()

In [14]:
category_df = pd.DataFrame(columns=['category', 'price'])
for category in unique_category:
    df = full_df[full_df['CATEGORY'] == category]
    del df['CATEGORY']
    category_df = category_df.append({
        'category': category, 
        'price': get_optimal_price(df)
    }, ignore_index=True)

In [16]:
category_df

,category,price
0,BAG SNACKS,1.430747
1,ORAL HYGIENE PRODUCTS,1.093578
2,COLD CEREAL,1.638499
3,FROZEN PIZZA,4.573120
